In [78]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/train.csv")
data = np.array(df)
m,n = data.shape
np.random.shuffle(data)

val_data = data[0 : 2000].T
Y_val = val_data[0]
X_val = val_data[1 : n]
X_val = X_val / 255.

train_data = data[2000 : m].T
Y_train = train_data[0]
X_train = train_data[1 : n]
X_train = X_train / 255.

In [72]:
def parameterss(hidden_layers, n_neurons, n_classes):
        weights = {}
        biases = {}

        for i in range(hidden_layers):

            if i == 0:
                #For the first hidden layer
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, 784) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5
            else:
                # For rest of the hidden layers
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, n_neurons) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5

        #For final output layer
        weights[f"W{hidden_layers + 1}"] = np.random.rand(n_classes, n_neurons) - 0.5
        biases[f"b{hidden_layers + 1}"] = np.random.rand(n_classes, 1) - 0.5

        return weights, biases

In [ ]:
w, b = parameterss(2,300,10)

In [ ]:
w.keys()

In [ ]:
b.keys()

In [ ]:
len(w)

In [ ]:
def ReLU(Z):
    return np.maximum(Z ,0)
    
    
def Softmax(Z):
    Z = np.float128(Z)
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def one_hot_encoding(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T

    return one_hot_Y

def derivative_ReLU(Z):
    return Z > 0

In [ ]:
def forward_propagation(weights, biases, X):
    l = len(weights)
    p = {}

    for i in range(1, l+1):
        if i == 1 :
            p[f"Z{i}"] = weights['W1'].dot(X) + biases['b1']
            p[f"A{i}"] = ReLU( weights['W1'].dot(X) + biases['b1'])
        elif i > 1 and i < l:
            p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
            p[f"A{i}"] = ReLU(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])
        elif i == l:
            p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
            p[f"A{i}"] = Softmax(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])

    return p

In [ ]:
a = forward_propagation(p1, p2, X_train)

In [ ]:
a.keys()

In [ ]:
def back_propagation(a, weights, X_train, Y_train):
    one_hot_Y = one_hot_encoding(Y_train)
    p = {}
    l = len(weights)

    for i in range(l, -1+1, -1):
        if i == l:
            p[f"dZ{i}"] = a[f"A{i}"] - one_hot_Y
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i < l and i > 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_ReLU(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i == 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_ReLU(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(X_train.T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)

    return  p

In [ ]:
s = back_propagation(a, w, X_train, Y_train)

In [ ]:
# l = 3
# for i in range(l,-1+1,-1):
#     print(i)

In [ ]:
s.keys()

In [ ]:
def update_parameters(weights, biases, s, alpha):
    p1 = {}
    p2 = {}

    l = len(weights)

    for i in range(1, l + 1):
        p1[f"W{i}"] = weights[f"W{i}"] - alpha * s[f"dW{i}"]
        p2[f"b{i}"] = biases[f"b{i}"] - alpha * s[f"db{i}"]

    return p1, p2

In [ ]:
p1, p2 = update_parameters(w, b, s, 0.1)

In [ ]:
p1.keys()

In [ ]:
p2.keys()

In [ ]:
def predictions(A):
    return np.argmax(A, axis = 0)
    

def accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [ ]:
last_element_key, last_element_value = list(a.items())[-1]
print(last_element_key)

In [ ]:
prediction = predictions(last_element_value)

In [ ]:
prediction

In [ ]:
accuracy(prediction, Y_train)